In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import classificacao

sns.set_style('darkgrid')
%matplotlib inline

In [3]:
df = pd.read_csv('conhecimento.csv')
df['dia'] = df.id_cena.str[-3:].astype(int)

id_cenas = [
    'LS82260702016078', 'LS82260702017256', # 226_070
    'LS82210702016059', 'LS82210702017269', # 221_070
    'LS52210672002068', 'LS82210672017285', # 221_067
    'LS82210742016059', 'LS82210742018128', # 221_074
]
col_sum = ['previsao', 'verdade', 'porcentagem', 'id_cena']

In [4]:
def gera_teste_modelo_caso1(id_cena):
    print(id_cena)

    teste_original = df[df.id_cena== id_cena].copy().reset_index(drop=True)
    teste = teste_original.copy()
    teste['verifica'] = 0
    print(len(teste_original))

    # Modelo
    modelo = df[~(df.id_cena==id_cena)].copy()

    print(len(modelo))
    print(modelo.verifica.value_counts())
    return teste, teste_original, modelo
def get_id_cenas(df, orb_pto):
    return sorted(list(df[df.orb_pto==orb_pto].id_cena.unique()))
def resultado(teste, teste_original, verifica=1):
    r = pd.merge(teste[['id', 'classificacao']], teste_original[['id', 'verifica']], on='id')
    r['v'] = r.classificacao.astype(str) + r.verifica.astype(str)
    
    try:
        x = len(r[(r.verifica==verifica) & (r.classificacao == verifica)])
        v = len(r[r.verifica==verifica])
        p = int((x*100)/v)
    except:
        return [0, 0, 0]
    return [x, v, p]

In [5]:
sumario = pd.DataFrame([], columns=['previsao', 'verdade', 'porcentagem', 'id_cena'])
for id_cena in id_cenas:
    teste, teste_original, modelo = gera_teste_modelo_caso1(id_cena)

    y_pred = classificacao.classifica(modelo, teste)
    y_true = teste_original.sort_values('id').reset_index(drop=True)

    r = resultado(y_pred, y_true)
    r.append(id_cena)
    sumario = sumario.append(dict(zip(col_sum, r)), ignore_index=True)

    print(r)    
#     classificacao.mostra_resultado(y_pred, y_true)
    #break

LS82260702016078
1894
386657
3    273474
1    113183
Name: verifica, dtype: int64
Criando classificador RF
RF - Classifica dado usando as colunas ['nbrl', 'medianb5']
Classificando com NN
Cria avaliacao usando (RF+NN)/2 - Arredondado para 3 casas decimais
[1, 1, 100, 'LS82260702016078']
LS82260702017256
961
387590
3    275211
1    112379
Name: verifica, dtype: int64
Criando classificador RF
RF - Classifica dado usando as colunas ['nbrl', 'medianb5']
Classificando com NN
Cria avaliacao usando (RF+NN)/2 - Arredondado para 3 casas decimais
[743, 805, 92, 'LS82260702017256']
LS82210702016059
2478
386073
3    272937
1    113136
Name: verifica, dtype: int64
Criando classificador RF
RF - Classifica dado usando as colunas ['nbrl', 'medianb5']
Classificando com NN
Cria avaliacao usando (RF+NN)/2 - Arredondado para 3 casas decimais
[18, 48, 37, 'LS82210702016059']
LS82210702017269
801
387750
3    275165
1    112585
Name: verifica, dtype: int64
Criando classificador RF
RF - Classifica dado usando

In [6]:
sumario

,previsao,verdade,porcentagem,id_cena
0,1,1,100,LS82260702016078
1,743,805,92,LS82260702017256
2,18,48,37,LS82210702016059
3,270,599,45,LS82210702017269
4,0,2,0,LS52210672002068
5,422,1102,38,LS82210672017285
6,0,1,0,LS82210742016059
7,252,1072,23,LS82210742018128


In [8]:
sumario.to_csv('sumario_caso1.csv', index=False)